In [4]:
from hyperopt import hp, tpe, Trials, fmin
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [5]:

# Define your dataset directory
dataset_dir ='dataset'

# Define image size and batch size
img_height, img_width = 640, 640
batch_size = 32

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting data into training and validation sets
)

# Load and prepare training data
train_data = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load and prepare validation data
val_data = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 1040 images belonging to 10 classes.
Found 255 images belonging to 10 classes.


In [6]:
# Define the search space for hyperparameters
space = {
    'optimizer': hp.choice('optimizer', [Adam, RMSprop, SGD]),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.1, 0.5)
}

# Define a function for training and evaluating the model
def train_model(params):
    optimizer = params['optimizer'](learning_rate=params['learning_rate'])
    
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dropout(params['dropout_rate']),
        layers.Dense(train_data.num_classes, activation='softmax')  # Use train_data.num_classes for dynamic class count
    ])

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Train the model
    history = model.fit(train_data, epochs=5, validation_data=val_data)
    
    # Get the validation accuracy
    val_acc = history.history['val_accuracy'][-1]
    
    return -val_acc  # Return negative accuracy since hyperopt minimizes the objective function

# Define the search space and optimization algorithm
trials = Trials()
best_params = fmin(train_model, space, algo=tpe.suggest, max_evals=10, trials=trials)

print("Best hyperparameters:", best_params)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

Epoch 1/5                                             

33/33 [==============================] - 406s 12s/step - loss: 2.3510 - accuracy: 0.1433 - val_loss: 2.1837 - val_accuracy: 0.1961

Epoch 2/5                                             

33/33 [==============================] - 248s 8s/step - loss: 2.1916 - accuracy: 0.1962 - val_loss: 2.0792 - val_accuracy: 0.2431

Epoch 3/5                                             

33/33 [==============================] - 246s 7s/step - loss: 2.2783 - accuracy: 0.1750 - val_loss: 2.2881 - val_accuracy: 0.1059

Epoch 4/5                                             

33/33 [==============================] - 242s 7s/step - loss: 2.2771 - accuracy: 0.1212 - val_loss: 2.2558 - val_accuracy: 0.1059

Epoch 5/5                                             

33/33 [==============================] - 244s 7s/step - loss: 2.2419 - accuracy: 0.1673 - val_loss: 2.1642 - val_accuracy: 0.2510

Epoch 1/5                                                       